In [1]:
import pandas as pd
import numpy as np

In [145]:
class Time:
    def __init__(self, name, c0, c1, h0, h1, penalty, demand, setupfee, isProduct0, cur_period_index):
        self.index = int(name)
        self.production_cost_0 = float(c0)
        self.production_cost_1 = float(c1)
        self.holding_cost_0 = float(h0)
        self.holding_cost_1 = float(h1)
        self.penalty = float(penalty)
        self.demand = float(demand)
        self.setupfee = float(setupfee)
        self.Product0 = bool(isProduct0)
        self.CurPeriodIndex = int(cur_period_index)
        self.cum_demand_for_production = {}
        self.cum_demand_for_holding = {}

    def __str__(self):
        return "time Index: " + str(self.index) + ", c0 = " + str(self.production_cost_0) + ", c1 = " + str(self.production_cost_1) + ", h0 = " + str(self.holding_cost_0) + ", h1 = " + str(self.holding_cost_1) + ", penalty = " + str(self.penalty) + ", demand = " + str(self.demand) + ", setupfee = " + str(self.setupfee) + ", can product 0 = " + str(self.Product0) + ", cur Time Period = " + str(self.CurPeriodIndex)

    def printCumDemandForProduct(self):
        print("Total demand for production at Time ", self.index, " is ", self.cum_demand_for_production)

    def printCumDemandForHolding(self):
        print("Total demand for holding at Time ", self.index, " is ", self.cum_demand_for_holding)

class TIME:
    def __init__(self, index, begin, end):
        self.index = index
        self.begin_time = begin
        self.end_time = end
        self.best_policy = {}   # key: TIME index, value: policy1 = -1, policy2 = -2, policy3 = positive number
        self.best_cost = {}   # key: TIME index, value: cost
        self.best = tuple()  # (cover to when, from when, policy num, policy cost)

    def __str__(self):
        return "TIME index: " + str(self.index) + ", begin_time = " + str(self.begin_time) + ", end_time = " + str(self.end_time)

    def printPolicy(self):
        key_list = list(self.best_policy.keys())
        for i in key_list:
            if self.best_policy[i] == -1:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: only order N95, and the cost is ", self.best_cost[i])
            elif self.best_policy[i] == -2:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: only order surgical mask, and the cost is ", self.best_cost[i])
            else:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: order N95 to cover time ", self.best_policy[i], ", and then order surgical mask, and the total cost is ", self.best_cost[i])

class PPE:
    def __init__(self, file_path, setupfee):
        self.TimeSeries = []
        self.TIMESeries = []
        self.product0_index = {}
        self.setUpFee = setupfee

        count = 0
        cur_T = -1
        with open(file_path, "r") as file:
            for line in file:
                if count == 0:
                    self.TotalTime = int(line.strip())
                elif count == 1:
                    self.Period = int(line.strip())
                else:
                    tmp_c0, tmp_c1, tmp_h0, tmp_h1, tmp_p, tmp_d, tmp_setup, tmp_product0 = line.strip().split(" ")

                    if tmp_product0 == "1":
                        cur_T += 1
                        self.product0_index[cur_T] = count - 2
                        tmp_TIME = TIME(cur_T, count - 2, count - 3 + self.Period)
                        self.TIMESeries.append(tmp_TIME)   # add big time period to TIME Series

                    tmp_t = Time(count - 2, tmp_c0, tmp_c1, tmp_h0, tmp_h1, tmp_p, tmp_d, tmp_setup, tmp_product0, cur_T)
                    last_t = self.TimeSeries[-1] if len(self.TimeSeries) > 0 else None   # record the previous one

                    self.TimeSeries.append(tmp_t)   # add small time period to Time Series

                    self.calculate_cumProductAndHoldingQuantity(tmp_t, last_t)   # update the cum demand for production and holding

                    # tmp_t.printCumDemandForProduct()
                    # tmp_t.printCumDemandForHolding()
                count += 1

        self.cal_best_policy()

    def backward(self, representInTIME):
        self.find_best_policy()
        tmp_index = len(self.TIMESeries) - 1
        tmp_best = []
        count = 0
        while (count < 100):
            if tmp_index < 0:
                break
            tmp_best.append(self.TIMESeries[tmp_index].best)
            tmp_index = self.TIMESeries[tmp_index].best[1] - 1
            count += 1

        for i in range(len(tmp_best)-1, -1, -1):
            if representInTIME:
                if (tmp_best[i][2] == -1):
                    print("Best policy for TIME ", tmp_best[i][1], "to TIME ", tmp_best[i][0], " is: only order N95, and the cost is ", tmp_best[i][3])
                elif (tmp_best[i][2] == -2):
                    print("Best policy for TIME ", tmp_best[i][1], "to TIME ", tmp_best[i][0], " is: only order surgical mask, and the cost is ", tmp_best[i][3])
                else:
                    print("Best policy for TIME ", tmp_best[i][1], "to TIME ", tmp_best[i][0], " is: order N95 to cover time ", tmp_best[i][2], ", and then order surgical mask, and the total cost is ", tmp_best[i][3])
            else:
                if (tmp_best[i][2] == -1):
                    print("Best policy for time ", self.TIMESeries[tmp_best[i][1]].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: only order N95, and the cost is ", tmp_best[i][3])
                elif (tmp_best[i][2] == -2):
                    print("Best policy for time ", self.TIMESeries[tmp_best[i][1]].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: only order surgical mask, and the cost is ", tmp_best[i][3])
                else:
                    print("Best policy for time ", self.TIMESeries[tmp_best[i][1]].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: order N95 to cover time ", tmp_best[i][2], ", and then order surgical mask, and the total cost is ", tmp_best[i][3])


    @staticmethod
    def calculate_cumProductAndHoldingQuantity(tmp_t, last_t):
        if tmp_t.index == 0:
            tmp_t.cum_demand_for_production[0] = tmp_t.demand
            tmp_t.cum_demand_for_holding[0] = 0
        else:
            for i in range(tmp_t.index):
                tmp_t.cum_demand_for_production[i] = last_t.cum_demand_for_production[i] + tmp_t.demand
                tmp_t.cum_demand_for_holding[i] = last_t.cum_demand_for_holding[i] + tmp_t.demand
            tmp_t.cum_demand_for_production[tmp_t.index] = tmp_t.demand
            tmp_t.cum_demand_for_holding[tmp_t.index] = 0

    def __len__(self):
        return self.TotalTime

    def __str__(self):
        print("TotalTime: " + str(self.TotalTime) + ", Period: " + str(self.Period))
        print("product0_index: ", self.product0_index)
        for t in self.TimeSeries:
            print(t)
        return ""

    # policy1: only order p0
    # policy2: only order p1
    # policy3: order p0 and p1 mix, need to find the best mix
    def cal_best_policy(self):
        for T in self.TIMESeries:
            for i in range(T.index, self.TIMESeries[-1].index + 1):
                T1 = self.TIMESeries[i]
                # print("In cal_best_policy, T = ", T.index, ", T1 = ", T1.index)
                policy1 = self.cal_policy1(T, T1, self.setUpFee)
                policy2 = self.cal_policy2(T, T1)
                policy3, best_time_policy3 = self.cal_policy3(T, T1, self.setUpFee)
                # print("Policy 1: ", policy1)
                # print("Policy 2: ", policy2)
                # print("Policy 3: ", policy3, " the best time is: ", best_time_policy3)
                best_policy  = min(policy1, policy2, policy3)   # minimal cost

                if best_policy == policy1:
                    T.best_policy[T1.index] = -1
                    T.best_cost[T1.index] = best_policy
                elif best_policy == policy2:
                    T.best_policy[T1.index] = -2
                    T.best_cost[T1.index] = best_policy
                else:
                    T.best_policy[T1.index] = best_time_policy3
                    T.best_cost[T1.index] = best_policy
            # T.printPolicy()   # check best policy

    def cal_policy1(self, tmp_T, tmp_T1, SetUpFee):
        produce_cost = 0
        hold_cost = 0
        for i in range(tmp_T.index, tmp_T1.index + 1):
            temp = self.TIMESeries[i]

            if SetUpFee:   # if setup fee is considered
                produce_cost += self.TimeSeries[temp.begin_time].setupfee   # setup fee

            produce_cost += self.TimeSeries[temp.begin_time].production_cost_0 * self.TimeSeries[temp.end_time].cum_demand_for_production[temp.begin_time]    # c_start * d(end, start)

            for j in range(temp.begin_time, temp.end_time + 1):
                hold_cost += self.TimeSeries[j].holding_cost_0 * self.TimeSeries[temp.end_time].cum_demand_for_holding[j]   # h_j * d(end, j)
        return produce_cost + hold_cost

    def cal_policy2(self, tmp_T, tmp_T1):
        produce_and_penalty_cost = 0
        for i in range(tmp_T.index, tmp_T1.index + 1):
            temp = self.TIMESeries[i]
            for i in range(temp.begin_time, temp.end_time + 1):
                produce_and_penalty_cost += (self.TimeSeries[i].production_cost_1 + self.TimeSeries[i].penalty) * self.TimeSeries[i].cum_demand_for_production[i]   # (c1 + p) * d(i, i)

        return produce_and_penalty_cost

    def cal_policy3(self, tmp_T, tmp_T1, SetUpFee):
        best_time = -1
        best_cost = 10000000000
        new_TIME_criteria = []   # [4, 8]
        for i in range(tmp_T.index, tmp_T1.index+1):
            new_TIME_criteria.append(self.product0_index[i])

        for i in range(tmp_T.begin_time, tmp_T1.end_time+1):
            temp_cost_production0 = 0
            temp_cost_holding0 = 0
            temp_production_and_penalty_cost = 0
            temp_total_cost = 0
            temp = self.TimeSeries[i]   # temp is small time period

            # print(i, " ", new_TIME_criteria)   # check setup fee
            if SetUpFee:   # if setup fee is considered
                for tempt in new_TIME_criteria:
                    if tempt <= temp.index:
                        temp_cost_production0 += self.TimeSeries[tempt].setupfee   # setup fee
            # print(i, " ", temp_cost_production0)   # check setup fee

            temp_cost_production0 += self.TimeSeries[tmp_T.begin_time].production_cost_0 * self.TimeSeries[i].cum_demand_for_production[tmp_T.begin_time]   # c_T's begin time * d(i, T's begin time)

            for j in range(tmp_T.begin_time, i + 1):
                temp_cost_holding0 += self.TimeSeries[j].holding_cost_0 * self.TimeSeries[i].cum_demand_for_holding[j]   # h_j * d(i, j)

            for j in range(i+1, tmp_T1.end_time + 1):
                temp_production_and_penalty_cost += (self.TimeSeries[j].production_cost_1 + self.TimeSeries[j].penalty) * self.TimeSeries[j].cum_demand_for_production[j]   # (c1 + p) * d(j, j)
            # print(temp_production_and_penalty_cost)

            temp_total_cost = temp_cost_production0 + temp_cost_holding0 + temp_production_and_penalty_cost

            if temp_total_cost <= best_cost:
                best_cost = temp_total_cost
                best_time = i

        return best_cost, best_time

    def find_best_policy(self):
        for tmp_T in range(len(self.TIMESeries)-1, -1, -1):
            cur_best = (tmp_T, tmp_T, self.TIMESeries[tmp_T].best_policy[tmp_T], self.TIMESeries[tmp_T].best_cost[tmp_T])
            for i in range(0, tmp_T):
                # print(i, self.TIMESeries[i].best_policy[tmp_T], self.TIMESeries[i].best_cost[tmp_T])
                if self.TIMESeries[i].best_cost[tmp_T] <= cur_best[2]:
                    cur_best = (tmp_T, i, self.TIMESeries[i].best_policy[tmp_T], self.TIMESeries[i].best_cost[tmp_T])
            self.TIMESeries[tmp_T].best = cur_best
            # print(cur_best)   # check the best policy


In [146]:
ppe = PPE("/Users/pikay/Documents/PPE/data/1.txt", True)   # True: setup fee, False: no setup fee

In [147]:
ppe.backward(False)  # True: TIME, False: time

Best policy for time  0 to time  3  is: order N95 to cover time  2 , and then order surgical mask, and the total cost is  1431.0
Best policy for time  4 to time  7  is: order N95 to cover time  6 , and then order surgical mask, and the total cost is  1341.0
Best policy for time  8 to time  11  is: order N95 to cover time  10 , and then order surgical mask, and the total cost is  1570.0
